In [1]:
%%time
import os
import numpy as np
os.chdir('../src/')
from util import clean_data as cd
from util import viz
import algos.basic as basic
import algos.similarity as sim
import pandas as pd
METADATA_PATH = '../data/Phenotypic_V1_0b_preprocessed1.csv'
PREFIX = '../data/rois_ez/'
POSTFIX = '_rois_ez.1D'

CPU times: user 692 ms, sys: 68 ms, total: 760 ms
Wall time: 1.34 s


In [2]:
%%time
md = cd.get_metadata(METADATA_PATH, ['FILE_ID', 'DX_GROUP'])

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 22.8 ms


In [3]:
%%time
good_md = cd.filter_file_names(md, PREFIX, POSTFIX)[:1]
f_names = np.array(map(lambda x: x['FILE_ID'], good_md))
dx_groups = np.array(map(lambda x: x['DX_GROUP'], good_md))

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 3.08 ms


In [4]:
%%time
data = cd.get_data(f_names)

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 13.8 ms


In [5]:
%%time
data = cd.run_function(basic.transpose, data)

Processed 0000 of 0001 brains.
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 156 µs


In [6]:
%%time
data = cd.run_function(basic.mean_center, data)
correl = cd.run_function(basic.correl, data)
viz.heatmap_channels(cd.run_function(basic.transpose, data),
         1,
         title='Activity over time',
         xtitle='Time step',
         ytitle='Region of Interest')
viz.heatmap_channels(correl,
         1,
         title='Correlation',
         xtitle='ROI',
         ytitle='ROI')

Processed 0000 of 0001 brains.
Processed 0000 of 0001 brains.
Processed 0000 of 0001 brains.


CPU times: user 180 ms, sys: 0 ns, total: 180 ms
Wall time: 181 ms


In [7]:
%%time
tsap = cd.run_function(sim.tsap, data)
n_clusts = cd.run_function(sim.n_clusts, tsap)
avg_clusts = cd.run_function(sim.avg_clust_size, tsap)
min_clusts = cd.run_function(sim.min_clust_size, tsap)
max_clusts = cd.run_function(sim.max_clust_size, tsap)
viz.line(n_clusts,
         1,
         title='Number of Clusters Over Time',
         xtitle='Time step',
         ytitle='Number of Clusters')
viz.line(avg_clusts,
         1,
         title='Average Cluster Size Over Time',
         xtitle='Time step',
         ytitle='Number of Clusters')
viz.line(max_clusts,
         1,
         title='Maximum Cluster Size Over Time',
         xtitle='Time step',
         ytitle='Number of Clusters')
viz.line(min_clusts,
         1,
         title='Minimum Cluster Size Over Time',
         xtitle='Time step',
         ytitle='Number of Clusters')

Processed 0000 of 0001 brains.
Processed 0000 of 0001 brains.
Processed 0000 of 0001 brains.
Processed 0000 of 0001 brains.
Processed 0000 of 0001 brains.


CPU times: user 15 s, sys: 144 ms, total: 15.2 s
Wall time: 14.9 s


In [8]:
viz.heatmap_channels(tsap,
                     1,
                     title='Clusters over Time',
                     xtitle='Timestep',
                     ytitle='ROI',
                     ztitle='Cluster Label')

In [9]:
%%time
dist = cd.run_function(sim.dtw, data)

Processed 0000 of 0001 brains.
CPU times: user 504 ms, sys: 0 ns, total: 504 ms
Wall time: 501 ms


In [10]:
viz.heatmap_channels(dist,
                     1,
                     title='Raw Data Heatmap',
                     xtitle='Timestep',
                     ytitle='ROI',
                     ztitle='4D Intensity (mean centered)')

In [11]:
%%time
affinity = cd.run_function(basic.neg, dist)
labels = cd.run_function(sim.ap, affinity)
ordered = cd.run_function(basic.cluster_sort, zip(data, labels))
correl = cd.run_function(basic.correl, ordered)

Processed 0000 of 0001 brains.
Processed 0000 of 0001 brains.


TypeError: 'numpy.int64' object is not iterable

In [ ]:
viz.heatmap_channels(correl,
                     1,
                     title='Raw Data Heatmap',
                     xtitle='Timestep',
                     ytitle='ROI',
                     ztitle='4D Intensity (mean centered)')

In [ ]:
%%time
sses = cd.run_function(basic.SSE, zip(data, labels))
ssses = [np.sum(s) for s in sses]

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(ssses, dx_groups)
plt.show()

In [ ]:
%%time
silhouette = cd.run_function(basic.silhouette, zip(dist, labels))
plt.scatter(silhouette, dx_groups)
plt.show()